In [39]:
import pandas as pd
from pydantic import BaseModel, Field
from typing import Dict, List, Optional, Any
import langgraph

In [40]:
class InfoProfile(BaseModel):
    name: str
    skills: List[str] = []
    experience: int = 0
    price: int = 0
    description: str = ""

class Jobdescription(BaseModel):
    post_title: str = ""
    post_description: str = ""
    post_skills_founded: List[str] = []	
    skills_asked_expliced: Optional[List[str]] = []
    level_asked: Optional[str] = ""

class ProposalState(BaseModel):
    headline: str = Field(..., description="Fist Line of the Proposal. Shuld get the attention of the client.")
    body: str = Field(..., description="Body of the Proposal. Should contain the details of the proposal.")
    full_text:str = Field(..., description="Full text of the Proposal.")
    

In [41]:
job_post_tile = """Client Trend Analysis and Credit Approval Automation"""
job_description = """Project Objectives
1. Client Trend Analysis
• Identify behavioral and demographic trends in our customer base.
• Group similar client profiles (segmentation/clustering).
• Predict client purchase behavior and potential lifetime value.
2. AI-Driven Credit Evaluation
• Analyze submitted application data (income, occupation, credit history, references, etc.).
• Integrate with internal/external data sources to enhance decision accuracy.
• Create a scoring model for automated or assisted credit approvals.
3. Dashboard & Integration
• Create a user-friendly dashboard to visualize trends and approve/reject applications.
• Optional: Integrate with our existing systems (CRM, Excel, SAP).
Scope of Work
• Data audit and ingestion
• AI/ML model development (client clustering, credit scoring)
• Model testing and validation
• Dashboard for credit review and trend insights
• Basic team training and documentation"""

In [42]:
profile_headline = """Data Scientist | LLM/Machine Learning | 10+ Million Saved Using AI"""
profile_summary = """
Struggling to make data-driven decisions that truly impact revenue? You know your business needs to leverage AI and machine learning, but you're unsure how to implement it effectively.
I specialize in transforming raw data into actionable insights that drive real financial results.
What people I work with say about me:

🏅"Italo is one of the most talented and impactful professionals I've worked with. He not only brings technical prowess but also a sharp business acumen that drives real value." – Igor Aboim, Staff Data Scientist
🏅"His expertise in building dashboards and managing data operations has consistently optimized our processes. A truly excellent professional." – Viviane Ghedini
🏅"I highly recommend Italo for his outstanding work in database management, performance analysis, and campaign optimization." – Marcus Plaisant

Here's what I've done for my clients and employers:

🤖 Multi-Agent AI System for Heineken Brazil: Developed a WhatsApp-based voice collection system that processes 2,000+ daily sales reports, analyzing 10,000+ hours of audio monthly to deliver real-time competitive intelligence across the entire country.
💰 Reduced absenteeism costs by $1M+/year using predictive modeling.
🔍 Developed AI-powered anomaly detection, saving $10M+ annually for Heineken Brazil.
📈 Increased e-commerce ROAS by 30% through Marketing Mix Modeling.
🤖 Designed and deployed Generative AI agents for HR automation and compliance validation.

We're a great fit if you're thinking:

💭 "We have data but don't know how to maximize its value."
💭 "We need an AI expert who understands business impact, not just algorithms."
💭 "I want scalable, automated solutions—not just reports and dashboards."
💭 "I don't have time to learn data science. I need someone to execute and deliver."

Working with me, you'll get:

🔮 Predictive models that increase efficiency and cut costs.
⚙️ AI-powered automation that eliminates manual work.
🔄 End-to-end solutions—from data pipeline setup to model deployment.
📊 AI-driven marketing insights to optimize ad spend and revenue.
My expertise includes:
🧠 Machine Learning & AI: Python, XGBoost, NLP, LLMs, LangChain, LangGraph, LLMops, Multi-agent systems
🔧 Data Engineering: SQL, Databricks, Google Cloud, Azure, BigQuery, Azure OpenAI
📊 Marketing Analytics: Marketing Mix Modeling, Campaign Optimization, Competitive Intelligence
📈 Business Intelligence: Power BI, Looker Studio, Automated Insights

Ready to unlock the full potential of your data? Click "Invite to Job" and let's discuss how I can help your business grow through AI and data science!
"""

protifolio = """
Multi-Agent AI: Processing 2K+ Daily Sales Reports for Heineken: 
Built a frictionless market intelligence solution for Heineken Brazil 
using WhatsApp voice notes from field representatives. Implemented a multi-agent AI 
system with LangGraph and Azure OpenAI that automatically extracts and structures 
competitive insights from audio recordings. Now processing inputs from 2,000+ reps and 
analyzing 10,000+ audio hours monthly, providing real-time.

Saved $10M+ with Advanced Anomaly Detection for Heineken Brazil:
Developed an advanced anomaly detection system for Heineken Brazil 
that automatically identifies root causes of financial losses across product/region combinations.
By implementing a two-layer approach combining statistical outlier detection and causal inference 
with SHAP values, I expanded analysis capacity from 25 to 600+ combinations monthly. 
This solution enabled rapid identification of loss drivers, resulting in 
$10M+ annual savings while providing actionable insights for business stakeholders.
"""



In [43]:
#load dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [45]:
api_key = os.getenv("GROQ_API_KEY")

In [46]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="qwen-2.5-32b",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [47]:
skills_needed = [
    "Generative AI expertise",
    "AI automation knowledge",
    "Content creation skills (for a 30-second commercial)",
    "Scriptwriting skills (for a catchy hook and commercial script)",
    "Understanding of AI applications in business (specifically lead generation and conversion)",
    "Familiarity with SMS and voice automation technologies",
    "Knowledge of ROI analysis and metrics (to understand 90% ROI increase)",
    "Creative problem-solving skills (to 'turn cold leads hot again')",
    "Collaboration and partnership skills (to work as a 'right-hand partner')",
    "Communication skills (to effectively convey the value proposition of the AI automation)"
]

pain_points = [
 "Low conversion rates of leads into booked appointments",
 "Difficulty in engaging with leads in a timely manner",
 "Loss of potential revenue due to leads slipping through the cracks",
 "Need to effectively communicate the value proposition of their AI automation solution to potential clients",
 "Desire to increase revenue and ROI for their business and their clients"
]

In [48]:
def get_skills(llm, job_description, job_post_tile):
    class SkillsFormater(BaseModel):
        skills: List[str] = []

    skill_stractor_prompt = f"""
    Bellow is a Upwork job description. 
    Extract Skills needed to perform the job from the following job description:

    <job_title>
    {job_post_tile}
    </job_title>

    <job_description>
    {job_description}
    </job_description>
    """
    skill_prompt = [
        ('system', skill_stractor_prompt),
        ('user', "return a python list of the skills needed")
    ]
    skills_structured_llm = llm.with_structured_output(SkillsFormater)
    skills_needed = skills_structured_llm.invoke(skill_prompt)
    return skills_needed

def get_pain_points(llm, job_description, job_post_tile):
    class PainPointsFormater(BaseModel):
        pain_points: List[str] = []
    
    client_motivations_prompt= f""" 
    Extract the client motivations from the following upwork job description:
    - Considere why the client is looking for a freelancer
    - Identify the main pain points that the client is facing that lead him to post a job.
    - Consider not only the pain points mentioned in the job description, but also the underlying issues that the client is facing.

    <job_title>
    {job_post_tile}
    </job_title>

    <job_description>
    {job_description}
    </job_description>
    """

    pain_prompt = [
        ('system', client_motivations_prompt),
        ('user', "return a python list of the pain points")
    ]
    pain_points_structured_llm = llm.with_structured_output(PainPointsFormater)
    pain_points = pain_points_structured_llm.invoke(pain_prompt)
    return pain_points

In [49]:
def selecting_arguments(llm, profile_headline, profile_summary, protifolio, pain_points, skills_needed):
    selecting_arguments_prompt = f"""
    Based on the my information prifile, select witch of the pain ponts and skills that I can help with.
    - Take into account my protifolio and my profile summary.
    - Analyze how I can help with the pain points and skills needed.
    - If a pain point is not related to my skills, ignore it.

    <profile>
        <profile_headline>
        {profile_headline}
        </profile_headline>
        <profile_summary>
        {profile_summary}
        </profile_summary>
        <protifolio>
        {protifolio}
        </protifolio>
    </profile>


    <pain_points>
    {pain_points}
    </pain_points>

    <skills_needed>
    {skills_needed}
    </skills_needed>

    
    """
    class SelectedArguments(BaseModel):
        arguments: List[Any] = Field(..., description="List of arguments showing how I can help with each pain point or skill")
    
    selected_arguments_prompt = [
        ('system', selecting_arguments_prompt),
        ('user', "Return a json of the pain points/skills that I can help with. The keys should be the pain points/skills and the values should be a brief description of how I can help with it. Be specific and persuasive. You can select exemples our of my protifolio.")
    ]
    selected_arguments_structured_llm = llm.with_structured_output(SelectedArguments)
    selected_arguments = selected_arguments_structured_llm.invoke(selected_arguments_prompt)
    return selected_arguments

In [55]:
def selecting_arguments(llm, profile_headline, profile_summary, protifolio, pain_points, skills_needed):
    class Argument(BaseModel):
        pain_point: str = Field(..., description="The client's pain point or required skill")
        solution: str = Field(..., description="How I can address this need based on my experience")

    class SelectedArguments(BaseModel):
        arguments: List[Argument] = Field(..., description="List of matching arguments between my experience and client needs")

    selecting_arguments_prompt = f"""
    Create specific matches between the client's needs and my experience.
    
    Profile Information:
    Headline: {profile_headline}
    Summary: {profile_summary}
    Portfolio: {protifolio}

    Client Needs:
    Pain Points: {pain_points}
    Required Skills: {skills_needed}

    For each key client need, provide:
    1. The specific pain point or skill needed
    2. How my experience directly addresses it, with concrete examples
    """
    
    selected_arguments_prompt = [
        ('system', selecting_arguments_prompt),
        ('user', """Return a list of arguments in this format:
        {
            "arguments": [
                {
                    "pain_point": "specific need",
                    "solution": "how I address it with specific example"
                },
                ...
            ]
        }
        Include 3-4 strongest matches.""")
    ]
    
    try:
        selected_arguments_structured_llm = llm.with_structured_output(SelectedArguments)
        return selected_arguments_structured_llm.invoke(selected_arguments_prompt)
    except Exception as e:
        print(f"Error in structured output: {str(e)}")
        # Return a default structured response
        return SelectedArguments(arguments=[
            Argument(
                pain_point="Error processing response",
                solution="Please try running the query again"
            )
        ])

In [50]:
skills = get_skills(llm, job_description, job_post_tile)
pain_points = get_pain_points(llm, job_description, job_post_tile)

In [51]:
pain_points.pain_points

['Lack of insight into client behavior and demographics',
 'Inefficient credit evaluation process',
 'Need for automated decision-making',
 'Data integration challenges',
 'Lack of user-friendly visualization tools',
 'Training and documentation needs']

In [52]:
skills.skills

['Data Analysis',
 'Machine Learning',
 'AI',
 'Predictive Modeling',
 'Credit Scoring',
 'Data Visualization',
 'Dashboard Creation',
 'Python Programming',
 'SQL',
 'Data Integration',
 'Clustering',
 'CRM Integration',
 'Excel Integration',
 'SAP Integration']

In [56]:
# Example usage:
arguments = selecting_arguments(llm, profile_headline, profile_summary, protifolio, pain_points.pain_points, skills.skills)

# Access the results
for arg in arguments.arguments:
    print(f"\nPain Point: {arg.pain_point}")
    print(f"Solution: {arg.solution}")


Pain Point: Lack of insight into client behavior and demographics
Solution: I developed an AI-powered anomaly detection system for Heineken Brazil that provides real-time insights into sales and market behavior, helping to identify trends and anomalies in client behavior and demographics.

Pain Point: Inefficient credit evaluation process
Solution: I have experience in predictive modeling and machine learning, which can be applied to create a more efficient credit scoring model. For instance, I used predictive modeling to reduce absenteeism costs by $1M+/year, demonstrating my ability to optimize processes through data-driven solutions.

Pain Point: Need for automated decision-making
Solution: I designed and deployed a multi-agent AI system for Heineken Brazil that automates the collection and analysis of sales reports, enabling real-time decision-making based on structured data inputs from 2,000+ field representatives.

Pain Point: Data integration challenges
Solution: I have extensi

In [54]:
arguments = selecting_arguments(llm, profile_headline, profile_summary, protifolio, pain_points.pain_points, skills.skills)

ValidationError: 1 validation error for SelectedArguments
arguments
  Input should be a valid list [type=list_type, input_value={'Lack of insight into cl...gs and revenue growth.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type

In [ ]:
arguments.arguments[0]

{'pain_point': 'Need to understand customer behavior and demographics to improve marketing and sales strategies.',
 'description': 'I can develop AI models using machine learning techniques to analyze customer behavior and demographics. By leveraging my expertise in Marketing Analytics and AI, I can create predictive models that identify key customer segments and their behaviors, enabling you to tailor marketing and sales strategies for maximum impact. For example, I have successfully used Marketing Mix Modeling to optimize ad spend and revenue for previous clients.'}